In [ ]:
!git clone https://github.com/yachty66/SimSwap.git
!cd SimSwap && git pull

In [ ]:
!pip install insightface==0.2.1 onnxruntime moviepy
!pip install googledrivedownloader
!pip install imageio==2.4.1

In [ ]:
import os
os.chdir("SimSwap")
!ls

In [ ]:
from google_drive_downloader import GoogleDriveDownloader

### it seems that google drive link may not be permenant, you can find this ID from our open url.
# GoogleDriveDownloader.download_file_from_google_drive(file_id='1TLNdIufzwesDbyr_nVTR7Zrx9oRHLM_N',
#                                     dest_path='./arcface_model/arcface_checkpoint.tar')
# GoogleDriveDownloader.download_file_from_google_drive(file_id='1PXkRiBUYbu1xWpQyDEJvGKeqqUFthJcI',
#                                     dest_path='./checkpoints.zip')

!wget -P ./arcface_model https://github.com/neuralchen/SimSwap/releases/download/1.0/arcface_checkpoint.tar
!wget https://github.com/neuralchen/SimSwap/releases/download/1.0/checkpoints.zip
!unzip ./checkpoints.zip  -d ./checkpoints
!wget -P ./parsing_model/checkpoint https://github.com/neuralchen/SimSwap/releases/download/1.0/79999_iter.pth

In [ ]:
!wget --no-check-certificate "https://sh23tw.dm.files.1drv.com/y4mmGiIkNVigkSwOKDcV3nwMJulRGhbtHdkheehR5TArc52UjudUYNXAEvKCii2O5LAmzGCGK6IfleocxuDeoKxDZkNzDRSt4ZUlEt8GlSOpCXAFEkBwaZimtWGDRbpIGpb_pz9Nq5jATBQpezBS6G_UtspWTkgrXHHxhviV2nWy8APPx134zOZrUIbkSF6xnsqzs3uZ_SEX_m9Rey0ykpx9w" -O antelope.zip
!unzip ./antelope.zip -d ./insightface_func/models/

In [ ]:
import cv2
import torch
import fractions
import numpy as npa
from PIL import Image
import torch.nn.functional as F
from torchvision import transforms
from models.models import create_model
from options.test_options import TestOptions
from insightface_func.face_detect_crop_multi import Face_detect_crop
from util.videoswap_multispecific import video_swap
import os
import glob

In [ ]:
def lcm(a, b): return abs(a * b) / fractions.gcd(a, b) if a and b else 0

transformer = transforms.Compose([
        transforms.ToTensor(),
        #transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

transformer_Arcface = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])


In [ ]:
!ls ./checkpoints

In [ ]:
!pip install opencv-python
!apt install cmake
!pip install face_recognition

In [ ]:
import os
import cv2
import numpy as np
import onnxruntime
import requests
from tqdm import tqdm
import face_recognition
from insightface_func.face_detect_crop_multi import Face_detect_crop  # Import the face detection class

# Initialize the face detection model
app = Face_detect_crop(name='antelope', root='./insightface_func/models')
app.prepare(ctx_id=0, det_thresh=0.6, det_size=(640, 640), mode='None')

def is_unique_face(face_encoding, known_faces, threshold=0.6):
    """Check if the face encoding is unique compared to known faces."""
    matches = face_recognition.compare_faces(known_faces, face_encoding, tolerance=threshold)
    return not any(matches)

def process_video(video_path, output_dir, margin):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise ValueError("Error: Could not open video.")

    os.makedirs(output_dir, exist_ok=True)
    known_faces = []
    face_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Use the face detection model to detect faces
        detect_results = app.get(frame, crop_size=640)  # Adjust crop_size as needed
        if detect_results is not None:
            frame_align_crop_list = detect_results[0]  # Get the aligned crops
            for frame_align_crop in frame_align_crop_list:
                face_img = frame_align_crop  # This is already the aligned face crop
                rgb_face_img = cv2.cvtColor(face_img, cv2.COLOR_BGR2RGB)

                # Get face encodings
                face_encodings = face_recognition.face_encodings(rgb_face_img)
                if face_encodings:
                    face_encoding = face_encodings[0]

                    if is_unique_face(face_encoding, known_faces):
                        known_faces.append(face_encoding)
                        face_count += 1
                        face_filename = os.path.join(output_dir, f'SRC_{face_count:02d}.jpg')
                        cv2.imwrite(face_filename, face_img, [int(cv2.IMWRITE_JPEG_QUALITY), 95])
                        print(f"Saved {face_filename}")

    cap.release()
    cv2.destroyAllWindows() 

# Example usage
if __name__ == "__main__":
    process_video('./demo_file/army.mp4', './demo_file/multispecific', margin=0.4)

In [ ]:
import sys
from options.test_options import TestOptions

# Temporarily clear sys.argv to avoid parsing any arguments
original_argv = sys.argv
sys.argv = [sys.argv[0]]  # Keep only the script name

# Create TestOptions and initialize
opt = TestOptions()
opt.initialize()
opt = opt.parse()  # This should return the parsed options

# Restore original sys.argv
sys.argv = original_argv

# Manually set or override the options
opt.multisepcific_dir = './demo_file/multispecific'
opt.video_path = './demo_file/output_720p.mp4'
opt.output_path = './output/out.mp4'
opt.temp_path = './tmp'
opt.Arc_path = './arcface_model/arcface_checkpoint.tar'
opt.name = 'people'
opt.isTrain = False
opt.use_mask = True
opt.crop_size = 224  # Set this to the default value or the one you need

# Add any other options that are required by your script
# opt.other_option = value

crop_size = opt.crop_size

# Print out the options to verify
print(opt)

In [ ]:
import os
import cv2
import torch
import glob
from PIL import Image
import torch.nn.functional as F
from models.models import create_model
from options.test_options import TestOptions
from insightface_func.face_detect_crop_multi import Face_detect_crop

# Assuming opt is already defined and initialized
pic_specific = opt.pic_specific_path
crop_size = opt.crop_size
multisepcific_dir = opt.multisepcific_dir

torch.nn.Module.dump_patches = True
model = create_model(opt)
model.eval()

app = Face_detect_crop(name='antelope', root='./insightface_func/models')
app.prepare(ctx_id=0, det_thresh=0.6, det_size=(640, 640))

# Directory to save images where face detection fails
error_dir = './error_images'
os.makedirs(error_dir, exist_ok=True)

# The specific person to be swapped (source)
source_specific_id_nonorm_list = []
source_path = os.path.join(multisepcific_dir, 'SRC_*')
source_specific_images_path = sorted(glob.glob(source_path))

with torch.no_grad():
    for source_specific_image_path in source_specific_images_path:
        specific_person_whole = cv2.imread(source_specific_image_path)
        if specific_person_whole is None:
            print(f"Error reading image: {source_specific_image_path}")
            continue

        result = app.get(specific_person_whole, crop_size)
        if result is None or len(result) != 2:
            print(f"Face detection failed for image: {source_specific_image_path}")
            error_image_path = os.path.join(error_dir, os.path.basename(source_specific_image_path))
            cv2.imwrite(error_image_path, specific_person_whole)
            continue

        specific_person_align_crop, _ = result
        specific_person_align_crop_pil = Image.fromarray(cv2.cvtColor(specific_person_align_crop[0], cv2.COLOR_BGR2RGB))
        specific_person = transformer_Arcface(specific_person_align_crop_pil)
        specific_person = specific_person.view(-1, specific_person.shape[0], specific_person.shape[1], specific_person.shape[2])
        specific_person = specific_person.cuda()
        specific_person_downsample = F.interpolate(specific_person, size=(112, 112))
        specific_person_id_nonorm = model.netArc(specific_person_downsample)
        source_specific_id_nonorm_list.append(specific_person_id_nonorm.clone())

# Print the count of successfully processed source images
print(f"Successfully processed source images: {len(source_specific_id_nonorm_list)}")

# The person who provides id information (list)
target_id_norm_list = []
target_path = os.path.join(multisepcific_dir, 'DST_*')
target_images_path = sorted(glob.glob(target_path))

with torch.no_grad():
    for target_image_path in target_images_path:
        img_a_whole = cv2.imread(target_image_path)
        if img_a_whole is None:
            print(f"Error reading image: {target_image_path}")
            continue

        result = app.get(img_a_whole, crop_size)
        if result is None or len(result) != 2:
            print(f"Face detection failed for image: {target_image_path}")
            error_image_path = os.path.join(error_dir, os.path.basename(target_image_path))
            cv2.imwrite(error_image_path, img_a_whole)
            continue

        img_a_align_crop, _ = result
        img_a_align_crop_pil = Image.fromarray(cv2.cvtColor(img_a_align_crop[0], cv2.COLOR_BGR2RGB))
        img_a = transformer_Arcface(img_a_align_crop_pil)
        img_id = img_a.view(-1, img_a.shape[0], img_a.shape[1], img_a.shape[2])
        img_id = img_id.cuda()
        img_id_downsample = F.interpolate(img_id, size=(112, 112))
        latend_id = model.netArc(img_id_downsample)
        latend_id = F.normalize(latend_id, p=2, dim=1)
        target_id_norm_list.append(latend_id.clone())

# Print the count of successfully processed target images
print(f"Successfully processed target images: {len(target_id_norm_list)}")

assert len(target_id_norm_list) == len(source_specific_id_nonorm_list), "The number of images in source and target directory must be same !!!"
video_swap(opt.video_path, target_id_norm_list, source_specific_id_nonorm_list, opt.id_thres, model, app, opt.output_path, temp_results_dir=opt.temp_path, no_simswaplogo=True, use_mask=opt.use_mask)

In [ ]:
%cd ..
%ls

In [ ]:
!DEBIAN_FRONTEND=noninteractive dpkg --configure -a

# Set DEBIAN_FRONTEND to noninteractive to avoid prompts
!DEBIAN_FRONTEND=noninteractive apt install git-all -y
!DEBIAN_FRONTEND=noninteractive apt install curl -y
!DEBIAN_FRONTEND=noninteractive apt-get install mesa-va-drivers -y

In [ ]:
!git clone https://github.com/yachty66/facefusion.git
%cd facefusion
!pip install -r requirements.txt
# Run the install and run scripts
!python install.py

In [ ]:
!python run.py --frame-processors face_enhancer -t ../SimSwap/output/out.mp4 -o ./enhanced_out.mp4 --face-enhancer-model gfpgan_1.4 --face-enhancer-blend 80 --face-selector-mode many --headless